<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/%5B20250317%5D%20Thesis_Hybrid%20RF-LSTM%20with%20Enhanced%20DeepFool%20Training%20on%20WSN-DS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Optimized Hybrid RF-LSTM with Enhanced DeepFool Adversarial Training on WSN-DS

In [1]:
!pip install adversarial-robustness-toolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.1 MB/s eta 0:00:00


In [2]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.6 MB/s eta 0:00:00


In [4]:
!pip install -U imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 4.9 MB/s eta 0:00:00


In [5]:
# Optimized Hybrid RF-LSTM with Enhanced DeepFool Adversarial Training

# Essential Imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from imblearn.over_sampling import SMOTE
from art.estimators.classification import TensorFlowV2Classifier
from art.attacks.evasion import DeepFool
from sklearn.metrics import accuracy_score
from tensorflow.keras.losses import CategoricalCrossentropy

In [6]:
# Load dataset
dataset = pd.read_csv("dataset_WSN-DS.csv")
dataset.dropna(inplace=True)

In [7]:
dataset

,id,Time,Is_CH,who CH,Dist_To_CH,ADV_S,ADV_R,JOIN_S,JOIN_R,SCH_S,SCH_R,Rank,DATA_S,DATA_R,Data_Sent_To_BS,dist_CH_To_BS,send_code,Expaned Energy,Class
0,101000,50,1,101000,0.00000,1,0,0,25,1,0,0,0,1200,48,130.08535,0,2.46940,0
1,101001,50,0,101044,75.32345,0,4,1,0,0,1,2,38,0,0,0.00000,4,0.06957,0
2,101002,50,0,101010,46.95453,0,4,1,0,0,1,19,41,0,0,0.00000,3,0.06898,0
3,101003,50,0,101044,64.85231,0,4,1,0,0,1,16,38,0,0,0.00000,4,0.06673,0
4,101004,50,0,101010,4.83341,0,4,1,0,0,1,25,41,0,0,0.00000,3,0.06534,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374656,201096,1003,0,201051,6.98337,0,5,1,0,0,1,7,96,0,67,170.14779,3,0.15974,0
374657,201097,1003,0,201037,29.32867,0,5,1,0,0,1,31,39,0,24,82.21043,2,0.06877,0
374658,201098,1003,0,201095,18.51963,0,5,1,0,0,1,17,55,0,31,139.26438,1,0.09437,0
374659,201099,1003,0,201051,8.55001,0,5,1,0,0,1,3,96,0,65,158.27492,3,0.16047,0


In [8]:
# Encode non-numeric columns (if any)
for column in dataset.columns:
    if dataset[column].dtype == 'object':  # Identify categorical columns
        dataset[column] = LabelEncoder().fit_transform(dataset[column])

# Separate features (X) and target (y)
X = dataset.drop(['Class'], axis=1)  # Assuming 'Class' is the target column
y = dataset['Class']

print("Original Class Distribution:", np.bincount(y))

Original Class Distribution: [340066  14596  10049   6638   3312]


In [9]:
from sklearn.preprocessing import LabelEncoder, StandardScaler # Import StandardScaler here
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [10]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the data
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Verify the balanced dataset distribution
print("Balanced Class Distribution:", np.bincount(y_resampled))

Balanced Class Distribution: [340066 340066 340066 340066 340066]


In [11]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=101, stratify=y_resampled)

In [12]:
# RF probability extraction
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

train_rf_probs = rf_model.predict_proba(X_train)
test_rf_probs = rf_model.predict_proba(X_test)

# Combine RF probabilities with original features
X_train_combined = np.concatenate([X_train, train_rf_probs], axis=1)
X_test_combined = np.concatenate([X_test, test_rf_probs], axis=1)

# Reshape for LSTM
X_train_combined = X_train_combined.reshape((X_train_combined.shape[0], 1, X_train_combined.shape[1]))
X_test_combined = X_test_combined.reshape((X_test_combined.shape[0], 1, X_test_combined.shape[1]))

In [13]:
# One-hot encode target
y_train_enc = to_categorical(y_train)
y_test_enc = to_categorical(y_test)

# Define LSTM model
lstm_model = Sequential([
    LSTM(64, input_shape=(X_train_combined.shape[1], X_train_combined.shape[2]), return_sequences=True, kernel_regularizer=l2(0.0001)),
    Dropout(0.2),
    LSTM(32, return_sequences=True, kernel_regularizer=l2(0.0001)),
    Dropout(0.2),
    LSTM(16, kernel_regularizer=l2(0.0001)),
    Dropout(0.2),
    Dense(y_train_enc.shape[1], activation='softmax')
])

lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train LSTM model
lstm_model.fit(X_train_combined, y_train_enc, epochs=30, batch_size=64, validation_split=0.2, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
17004/17004 ━━━━━━━━━━━━━━━━━━━━ 71s 4ms/step - accuracy: 0.9864 - loss: 0.0930 - val_accuracy: 0.9999 - val_loss: 0.0030
Epoch 2/30
17004/17004 ━━━━━━━━━━━━━━━━━━━━ 66s 4ms/step - accuracy: 0.9999 - loss: 0.0030 - val_accuracy: 1.0000 - val_loss: 0.0022
Epoch 3/30
17004/17004 ━━━━━━━━━━━━━━━━━━━━ 69s 4ms/step - accuracy: 0.9999 - loss: 0.0027 - val_accuracy: 1.0000 - val_loss: 0.0017
Epoch 4/30
17004/17004 ━━━━━━━━━━━━━━━━━━━━ 69s 4ms/step - accuracy: 0.9999 - loss: 0.0026 - val_accuracy: 1.0000 - val_loss: 0.0016
Epoch 5/30
17004/17004 ━━━━━━━━━━━━━━━━━━━━ 68s 4ms/step - accuracy: 0.9999 - loss: 0.0022 - val_accuracy: 1.0000 - val_loss: 0.0019
Epoch 6/30
17004/17004 ━━━━━━━━━━━━━━━━━━━━ 68s 4ms/step - accuracy: 0.9999 - loss: 0.0022 - val_accuracy: 1.0000 - val_loss: 0.0020
Epoch 7/30
17004/17004 ━━━━━━━━━━━━━━━━━━━━ 64s 4ms/step - accuracy: 0.9999 - loss: 0.0021 - val_accuracy: 1.0000 - val_loss: 0.0019
Epoch 8/30
17004/17004 ━━━━━━━━━━━━━━━━━━━━ 62s 4ms/step - accuracy: 

In [14]:
# prompt: Display model accuracy, precision, Recall, and F1-Score

from sklearn.metrics import precision_score, recall_score, f1_score

# ... (Your existing code) ...

# Make predictions
y_pred_prob = lstm_model.predict(X_test_combined)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test_enc, axis=1)

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted') # Use 'weighted' for multi-class
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')


print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

10628/10628 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step
Accuracy: 0.9991531055736239
Precision: 0.9991530029092434
Recall: 0.9991531055736239
F1-Score: 0.9991529374162764


In [15]:
from art.attacks.evasion import DeepFool  # Ensure this line is present

# ART classifier setup
classifier = TensorFlowV2Classifier(
    model=lstm_model,
    nb_classes=y_train_enc.shape[1],
    input_shape=(X_train_combined.shape[1], X_train_combined.shape[2]),
    loss_object=CategoricalCrossentropy()
)

# Select a smaller subset for faster adversarial example generation (optional, but recommended)
subset_size = min(500, X_train_combined.shape[0])  # use at most 500 samples
X_train_combined_subset = X_train_combined[:subset_size]
y_test_subset = y_test[:subset_size]

# DeepFool Attack Generation (lighter and faster)
deepfool_attack = DeepFool(
    classifier=classifier,
    max_iter=20,               # Significantly reduced from 20
    epsilon=1e-4,             # Increased epsilon for faster convergence
    batch_size=8             # Increased batch size
)

X_test_adv_deepfool = deepfool_attack.generate(X_train_combined_subset)

# Evaluate accuracy under DeepFool attack
y_pred_deepfool = np.argmax(classifier.predict(X_test_adv_deepfool), axis=1)
accuracy_deepfool = accuracy_score(y_test_subset, y_pred_deepfool)
print(f"Accuracy under DeepFool attack: {accuracy_deepfool:.10f}")

DeepFool:   0%|          | 0/63 [00:00<?, ?it/s]

Accuracy under DeepFool attack: 0.2260000000
